# VAZHI SFT v4.1 — Instruction Fine-Tuning on DAPT v1.1

**Pipeline Step 3 of 3:** SFT on the DAPT-adapted Tamil instruct model.

```
Step 1: Data Prep (DONE — Vazhi_DAPT_Data_v1_1.ipynb)
Step 2: DAPT Training (DONE — Vazhi_DAPT_v1_1_Tamil.ipynb)
  → Produced: CryptoYogi/qwen3-0.6b-tamil-v1_1
    PPL 2.6, +55% Tamil vs vanilla, 7/8 eval passed

Step 2.5: Dataset Factory v4.1.3 (DONE — Vazhi_Dataset_Factory_v4_1_3.ipynb)
  → Produced: CryptoYogi/vazhi-tamil-sft-v4_1 (14,535 samples)

Step 3 (THIS NOTEBOOK): SFT — teach instruction-following in Tamil
  → Input:  DAPT'd model + v4.1 ChatML dataset (14,535 samples)
  → Output: CryptoYogi/vazhi-v4_1 (final VAZHI model)
           CryptoYogi/vazhi-v4_1-lora (adapter backup)
```

## v4.0 vs v4.1 Comparison

| Parameter | v4.0 (FAILED) | v4.1 |
|-----------|---------------|------|
| Train samples | 1,365 | **13,083** (10x) |
| LoRA r | 16 | **8** |
| Target modules | 7 (all proj) | **2 (q_proj, v_proj)** |
| Epochs | 3 | **2** |
| LR | 2e-5 | **5e-5** |
| max_seq_length | 1024 | **2048** |
| GPU | Kaggle T4 x2 | **Colab Pro L4** |
| Dtype | fp16 | **bf16 (auto-detected)** |
| Think suppression | suppress_tokens kwarg (broken) | **Custom LogitsProcessor** |
| Eval | Tamil % only (false positives) | **Conversational quality (fluency, intent-matching, no gibberish)** |
| Hub checkpoint | No | **Yes (every save_steps)** |

**v4.0 failure root causes:** LoRA r=16 on 7 modules overfit 1,365 samples; 3 epochs = memorization;
max_seq_length=1024 rejected 74% domain packs; automated eval gave false positives (12/12 'passed' but gibberish).

**Eval philosophy:** The model is NOT a knowledge base — factual lookups are handled by the
hybrid architecture (SQLite). SFT eval tests conversational quality: Tamil fluency, instruction-following,
appropriate tone, safety refusals, and coherent responses. NOT factual recall.

**Target:** Colab Pro L4 | ~3,270 steps (2 epochs) | Est. 30-45 min

In [ ]:
# Cell 1: Dependencies
# After running this cell, RESTART the session (Runtime → Restart session)

!pip install -q -U \
  "transformers>=4.45.0,<5.0.0" \
  "accelerate>=0.34.2" \
  "peft>=0.12.0" \
  "trl>=0.12.0,<0.20.0" \
  "datasets>=2.21.0" \
  "huggingface_hub>=0.24.7"

print("\u2705 Dependencies installed")
print("\u26a0\ufe0f  RESTART THE SESSION NOW (Runtime \u2192 Restart session)")

In [ ]:
# Cell 2: Config + GPU Auto-Detection

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import json
import re
import random
import glob
import gc
import shutil
import hashlib
import torch
import numpy as np
from collections import Counter
from datasets import load_dataset
from huggingface_hub import login, HfApi

from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    TrainerCallback, LogitsProcessorList,
)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# === KEY CONFIG ===
DAPT_MODEL = "CryptoYogi/qwen3-0.6b-tamil-v1_1"   # DAPT'd instruct (Step 2 output)
VANILLA_MODEL = "Qwen/Qwen3-0.6B"                  # For pre-SFT baseline comparison
SFT_DATASET = "CryptoYogi/vazhi-tamil-sft-v4_1"    # v4.1 ChatML dataset (14,535 samples)
DAPT_DATASET = "CryptoYogi/vazhi-dapt-tamil-v1_1"  # For perplexity baseline
OUTPUT_MODEL = "CryptoYogi/vazhi-v4_1"              # Final VAZHI model
ADAPTER_REPO = "CryptoYogi/vazhi-v4_1-lora"         # Adapter backup

# Training config (v4.1 fixes: conservative LoRA, more data, fewer epochs)
LEARNING_RATE = 5e-5       # Higher than v4.0 (2e-5) for stronger instruction signal
NUM_EPOCHS = 2             # 2 not 3 — 10x data means 2 epochs is enough
MAX_SEQ_LENGTH = 2048      # v4.0 used 1024, rejected 74% domain packs
LORA_R = 8                 # v4.0 used 16 — r=8 avoids overfitting
LORA_ALPHA = 16            # Standard 2x ratio
BATCH_SIZE = 4             # Per-device (L4 has 22GB)
GRADIENT_ACCUMULATION = 2  # 4 x 1 GPU x 2 = 8 effective batch

# Qwen3 instruct <think> tokens to suppress during generation
THINK_TOKEN_IDS = [151667, 151668]

SYSTEM_PROMPT = (
    "\u0ba8\u0bc0\u0b99\u0bcd\u0b95\u0bb3\u0bcd VAZHI (\u0bb5\u0bb4\u0bbf), \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0bae\u0b95\u0bcd\u0b95\u0bb3\u0bc1\u0b95\u0bcd\u0b95\u0bbe\u0ba9 AI \u0b89\u0ba4\u0bb5\u0bbf\u0baf\u0bbe\u0bb3\u0bb0\u0bcd. "
    "\u0ba4\u0bae\u0bbf\u0bb4\u0bbf\u0bb2\u0bcd \u0ba4\u0bc6\u0bb3\u0bbf\u0bb5\u0bbe\u0b95\u0bb5\u0bc1\u0bae\u0bcd \u0b89\u0ba4\u0bb5\u0bbf\u0baf\u0bbe\u0b95\u0bb5\u0bc1\u0bae\u0bcd \u0baa\u0ba4\u0bbf\u0bb2\u0bb3\u0bbf\u0baf\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd. "
    '\u0ba4\u0bc6\u0bb0\u0bbf\u0baf\u0bbe\u0bb5\u0bbf\u0b9f\u0bcd\u0b9f\u0bbe\u0bb2\u0bcd "\u0ba4\u0bc6\u0bb0\u0bbf\u0baf\u0bb5\u0bbf\u0bb2\u0bcd\u0bb2\u0bc8" \u0b8e\u0ba9\u0bcd\u0bb1\u0bc1 \u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd.'
)

# GPU auto-detection (from Dataset Factory v4.1.2)
assert torch.cuda.is_available(), "GPU required! Runtime > Change runtime type > GPU"
gpu_name = torch.cuda.get_device_name(0).lower()
VRAM_GB = torch.cuda.get_device_properties(0).total_memory / 1e9
IS_HIGH_END_GPU = any(x in gpu_name for x in ["a100", "l4", "h100", "a10"])
USE_BF16 = IS_HIGH_END_GPU  # bf16 on L4/A100, fp16 on T4
MODEL_DTYPE = torch.bfloat16 if USE_BF16 else torch.float16
n_gpus = torch.cuda.device_count()

effective_batch = BATCH_SIZE * n_gpus * GRADIENT_ACCUMULATION

print(f"\u2705 Configuration loaded")
print(f"   PyTorch: {torch.__version__}")
print(f"   GPU: {torch.cuda.get_device_name(0)} ({VRAM_GB:.0f} GB)")
print(f"   Tier: {'high-end' if IS_HIGH_END_GPU else 'standard'}")
print(f"   Dtype: {'bf16' if USE_BF16 else 'fp16'}")
print(f"   GPUs: {n_gpus}")
print()
print(f"\U0001f4cb SFT v4.1 on DAPT v1.1:")
print(f"   Base:     {DAPT_MODEL} (DAPT'd instruct)")
print(f"   Dataset:  {SFT_DATASET}")
print(f"   Output:   {OUTPUT_MODEL}")
print(f"   LR:       {LEARNING_RATE}")
print(f"   LoRA:     r={LORA_R}, alpha={LORA_ALPHA}, targets=[q_proj, v_proj]")
print(f"   Batch:    {BATCH_SIZE} x {n_gpus} GPU x {GRADIENT_ACCUMULATION} accum = {effective_batch} effective")
print(f"   Epochs:   {NUM_EPOCHS}")
print(f"   Seq len:  {MAX_SEQ_LENGTH}")
print(f"   dtype:    {'bf16' if USE_BF16 else 'fp16'}")

In [ ]:
# Cell 3: HuggingFace Login (platform-agnostic)

try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    hf_token = secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("\u2705 Logged in via Kaggle secrets")
except Exception:
    try:
        from google.colab import userdata
        hf_token = userdata.get('HF_TOKEN')
        login(token=hf_token)
        print("\u2705 Logged in via Colab secrets")
    except Exception:
        login()
        print("\u2705 Logged in interactively")

In [ ]:
# Cell 4: Pre-SFT Perplexity Baseline
# Compare vanilla instruct vs DAPT'd model on Tamil validation blocks.
# Hard abort if DAPT regressed.

print("\U0001f4ca Pre-SFT Validation: Perplexity Baseline")
print("=" * 60)

print(f"\U0001f4e5 Loading Tamil val set from {DAPT_DATASET}...")
dapt_ds = load_dataset(DAPT_DATASET, split="validation")
n_eval = min(100, len(dapt_ds))
print(f"   {len(dapt_ds)} val blocks available, using {n_eval}")


def compute_ppl(model, dataset, n_samples, device):
    """Compute perplexity on pre-tokenized blocks."""
    model.eval()
    losses = []
    for i in range(n_samples):
        input_ids = torch.tensor([dataset[i]["input_ids"]], dtype=torch.long).to(device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=input_ids.clone())
            losses.append(outputs.loss.item())
    avg_loss = np.mean(losses)
    return np.exp(min(avg_loss, 20)), avg_loss


# 1. Vanilla instruct PPL
print(f"\n\U0001f4e5 Loading vanilla {VANILLA_MODEL}...")
vanilla = AutoModelForCausalLM.from_pretrained(
    VANILLA_MODEL, torch_dtype=MODEL_DTYPE, device_map={"":0}, trust_remote_code=True,
)
vanilla_ppl, vanilla_loss = compute_ppl(vanilla, dapt_ds, n_eval, vanilla.device)
print(f"   Vanilla PPL: {vanilla_ppl:.2f} (loss: {vanilla_loss:.4f})")
del vanilla; gc.collect(); torch.cuda.empty_cache()

# 2. DAPT'd model PPL
print(f"\n\U0001f4e5 Loading DAPT'd {DAPT_MODEL}...")
dapt = AutoModelForCausalLM.from_pretrained(
    DAPT_MODEL, torch_dtype=MODEL_DTYPE, device_map={"":0}, trust_remote_code=True,
)
dapt_ppl, dapt_loss = compute_ppl(dapt, dapt_ds, n_eval, dapt.device)
print(f"   DAPT PPL:    {dapt_ppl:.2f} (loss: {dapt_loss:.4f})")

# Compare
ppl_improvement = vanilla_ppl - dapt_ppl
ppl_pct = 100 * (vanilla_ppl - dapt_ppl) / vanilla_ppl
print(f"\n\U0001f4ca PPL Comparison:")
print(f"   Vanilla: {vanilla_ppl:.2f}")
print(f"   DAPT:    {dapt_ppl:.2f}")
print(f"   Change:  {ppl_improvement:+.2f} ({ppl_pct:+.1f}%)")

if dapt_ppl < vanilla_ppl:
    print(f"\n\u2705 DAPT improved Tamil perplexity! Safe to proceed with SFT.")
elif dapt_ppl < vanilla_ppl * 1.05:
    print(f"\n\u26a0\ufe0f  DAPT is neutral (within 5%). Proceeding with caution.")
else:
    raise RuntimeError(
        f"\u274c HARD ABORT: DAPT made perplexity WORSE "
        f"({dapt_ppl:.2f} > {vanilla_ppl:.2f}). Do NOT proceed with SFT."
    )

In [ ]:
# Cell 5: Chat Template Test + All Helper Functions
#
# Defines ALL generation helpers in one place (reused from v4.0 + Eval v4.0):
# - SuppressThinkTokens (custom LogitsProcessor — NOT the broken suppress_tokens kwarg)
# - build_chat_prompt(), strip_think_tags(), extract_response()
# - tamil_char_pct(), compute_repeat_ratio()

print("\U0001f9ea Setting up helpers + chat template test")
print("=" * 60)

tokenizer = AutoTokenizer.from_pretrained(DAPT_MODEL, trust_remote_code=True)

# Check if tokenizer supports enable_thinking
try:
    test = tokenizer.apply_chat_template(
        [{"role": "user", "content": "test"}],
        tokenize=False, add_generation_prompt=True, enable_thinking=False,
    )
    USE_THINKING_FLAG = True
    print("\u2705 Tokenizer supports enable_thinking=False")
except TypeError:
    USE_THINKING_FLAG = False
    print("\u26a0\ufe0f  enable_thinking not supported, using manual template")


# --- Custom LogitsProcessor for <think> suppression ---
# The suppress_tokens kwarg in generate() has a CPU/CUDA device mismatch bug
# in transformers. This custom processor handles it correctly.
class SuppressThinkTokens:
    """Suppress specific token IDs by setting their logits to -inf."""
    def __init__(self, token_ids, device):
        self.suppress_ids = torch.tensor(token_ids, dtype=torch.long, device=device)

    def __call__(self, input_ids, scores):
        scores[:, self.suppress_ids] = float('-inf')
        return scores


def build_chat_prompt(user_text):
    msgs = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_text},
    ]
    if USE_THINKING_FLAG:
        return tokenizer.apply_chat_template(
            msgs, tokenize=False, add_generation_prompt=True, enable_thinking=False,
        )
    else:
        return (
            f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
            f"<|im_start|>user\n{user_text}<|im_end|>\n"
            f"<|im_start|>assistant\n"
        )


def strip_think_tags(text):
    """Remove <think>...</think> blocks (belt & suspenders fallback)."""
    text = re.sub(r'<think>.*?</think>\s*', '', text, flags=re.DOTALL)
    text = re.sub(r'</?think>', '', text)
    return text.strip()


def extract_response(full_text):
    """Extract assistant response, stripping think tags."""
    if "<|im_start|>assistant" in full_text:
        resp = full_text.split("<|im_start|>assistant")[-1]
        resp = resp.split("<|im_end|>")[0].strip()
        if resp.startswith("\n"):
            resp = resp[1:]
    else:
        resp = full_text
    return strip_think_tags(resp)


def count_tamil_chars(text):
    return sum(1 for c in text if '\u0B80' <= c <= '\u0BFF')


def tamil_char_pct(text):
    if not text:
        return 0.0
    return 100.0 * count_tamil_chars(text) / len(text)


def compute_repeat_ratio(text, n=3):
    """Fraction of tokens in repeated n-gram chains. >0.2 is bad."""
    words = text.split()
    if len(words) < n:
        return 0.0
    ngrams = [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]
    seen = set()
    repeated_positions = set()
    for i, ng in enumerate(ngrams):
        if ng in seen:
            for j in range(i, i + n):
                repeated_positions.add(j)
        seen.add(ng)
    return len(repeated_positions) / max(len(words), 1)


print("\u2705 Helper functions defined")

# --- Chat template test on DAPT model ---
# Verify DAPT'd model still produces Tamil before we invest in SFT

# Clear suppress_tokens from generation_config to prevent buggy built-in processor
if hasattr(dapt, 'generation_config') and hasattr(dapt.generation_config, 'suppress_tokens'):
    dapt.generation_config.suppress_tokens = None
    print("\U0001f527 Cleared suppress_tokens from DAPT generation_config")

think_suppressor = SuppressThinkTokens(THINK_TOKEN_IDS, dapt.device)
logits_procs = LogitsProcessorList([think_suppressor])

dapt.eval()
dapt.config.use_cache = True

chat_prompts = [
    "\u0bb5\u0ba3\u0b95\u0bcd\u0b95\u0bae\u0bcd",
    "\u0ba4\u0bae\u0bbf\u0bb4\u0bcd\u0ba8\u0bbe\u0b9f\u0bcd\u0b9f\u0bbf\u0ba9\u0bcd \u0ba4\u0bb2\u0bc8\u0ba8\u0b95\u0bb0\u0bae\u0bcd \u0b8e\u0ba9\u0bcd\u0ba9?",
    "\u0ba8\u0bc0\u0b99\u0bcd\u0b95\u0bb3\u0bcd \u0baf\u0bbe\u0bb0\u0bcd?",
]

for prompt_text in chat_prompts:
    full_prompt = build_chat_prompt(prompt_text)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(dapt.device)
    with torch.no_grad():
        outputs = dapt.generate(
            **inputs, max_new_tokens=100, do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            logits_processor=logits_procs,
        )
    full = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = extract_response(full)
    t_pct = tamil_char_pct(response)
    print(f"\n  Q: {prompt_text}")
    print(f"  A: {response[:200]}")
    print(f"  Tamil: {t_pct:.0f}%")

print("\n" + "=" * 60)
print("If responses are Tamil (even if incoherent), DAPT preserved")
print("language capability. SFT will teach instruction-following.")

# Free DAPT model and val set
del dapt, dapt_ds, think_suppressor, logits_procs
gc.collect(); torch.cuda.empty_cache()
print("\n\U0001f5d1\ufe0f Pre-validation models freed")

In [ ]:
# Cell 6: Load & Validate SFT Dataset

print(f"\U0001f4da Loading SFT dataset from {SFT_DATASET}...")
sft_ds = load_dataset(SFT_DATASET)
train_ds = sft_ds["train"]
eval_ds = sft_ds["validation"]

print(f"\u2705 Dataset loaded:")
print(f"   Train:      {len(train_ds)} samples")
print(f"   Validation: {len(eval_ds)} samples")
print(f"   Columns:    {train_ds.column_names}")

# Composition stats
bucket_dist = Counter(item.get('bucket', 'unknown') for item in train_ds)
print(f"\n\U0001f4ca Composition:")
for bucket, count in sorted(bucket_dist.items(), key=lambda x: -x[1]):
    print(f"   {bucket}: {count} ({100*count/len(train_ds):.1f}%)")

# ChatML validation (all samples) — hard abort if >1% fail
CHATML_RE = re.compile(
    r'<\|im_start\|>system\n.+?<\|im_end\|>\n'
    r'<\|im_start\|>user\n(.+?)<\|im_end\|>\n'
    r'<\|im_start\|>assistant\n(.+?)<\|im_end\|>',
    re.DOTALL
)

fail_count = 0
think_count = 0
for i in range(len(train_ds)):
    text = train_ds[i]["text"]
    if not CHATML_RE.search(text):
        fail_count += 1
        if fail_count <= 3:
            print(f"   \u274c Sample {i}: invalid ChatML")
    if "<think>" in text or "</think>" in text:
        think_count += 1
        if think_count <= 3:
            print(f"   \u26a0\ufe0f Sample {i}: contains <think> tag!")

fail_pct = 100 * fail_count / len(train_ds)
if fail_count == 0:
    print(f"\n\u2705 All {len(train_ds)} train samples pass ChatML validation")
else:
    print(f"\n\u274c {fail_count} samples failed ChatML ({fail_pct:.1f}%)")
    if fail_pct > 1.0:
        raise RuntimeError(f"HARD ABORT: {fail_pct:.1f}% ChatML failures (>1% threshold)")

if think_count == 0:
    print(f"\u2705 No <think> tags in training data")
else:
    print(f"\u26a0\ufe0f {think_count} samples contain <think> tags")

# Show a sample
m = CHATML_RE.search(train_ds[0]["text"])
if m:
    print(f"\n\U0001f50d Sample:")
    print(f"   Q: {m.group(1)[:100]}")
    print(f"   A: {m.group(2)[:150]}")

In [ ]:
# Cell 7: Load Tokenizer + Model
#
# Load DAPT'd model in auto-detected dtype.
# NO device_map — use .to("cuda:0") to avoid breaking Trainer's DataParallel.
# Disable cache + enable gradient checkpointing for training.

print(f"\U0001f4e5 Loading tokenizer from {DAPT_MODEL}...")
tokenizer = AutoTokenizer.from_pretrained(DAPT_MODEL, trust_remote_code=True)
tokenizer.padding_side = "right"

print(f"\u2705 Tokenizer: {len(tokenizer)} tokens")
print(f"   eos: {tokenizer.eos_token!r} (ID {tokenizer.eos_token_id})")
print(f"   pad: {tokenizer.pad_token!r} (ID {tokenizer.pad_token_id})")

# Verify ChatML tokens
for token in ["<|im_start|>", "<|im_end|>"]:
    assert token in tokenizer.get_vocab(), f"Missing {token}!"
print("\u2705 ChatML tokens present")

# Verify <think> tokens
for tid in THINK_TOKEN_IDS:
    print(f"   Token {tid}: {tokenizer.decode([tid])!r}")

# Load model — NO device_map for training
dtype_str = 'bf16' if USE_BF16 else 'fp16'
print(f"\n\U0001f4e5 Loading {DAPT_MODEL} in {dtype_str}...")
print(f"   NO device_map \u2014 Trainer handles device placement")

model = AutoModelForCausalLM.from_pretrained(
    DAPT_MODEL,
    torch_dtype=MODEL_DTYPE,
    trust_remote_code=True,
)
model = model.to("cuda:0")

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.use_cache = False
model.gradient_checkpointing_enable()

# Verify no hf_device_map (would prevent DataParallel)
has_dm = hasattr(model, "hf_device_map")
print(f"   hf_device_map: {has_dm} (must be False)")
if has_dm:
    print(f"   \u26a0\ufe0f WARNING: hf_device_map detected!")

mem_gb = torch.cuda.memory_allocated(0) / 1024**3
print(f"\u2705 Model loaded: {model.num_parameters():,} params ({mem_gb:.1f} GB)")

In [ ]:
# Cell 8: LoRA Setup
#
# v4.1 fix: r=8 on q_proj+v_proj only (v4.0 used r=16 on 7 modules → overfitting)

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],  # v4.0 targeted all 7 → overfit
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

mem_gb = torch.cuda.memory_allocated() / 1024**3
print(f"\u2705 LoRA applied | GPU: {mem_gb:.1f} GB")

In [ ]:
# Cell 9: Completion-Only Masking
#
# Only train on assistant responses. System prompt and user messages are masked (-100).
# Preflight verify on 20 samples.

response_template_str = "<|im_start|>assistant\n"
response_template_ids = tokenizer.encode(response_template_str, add_special_tokens=False)
print(f"Response template: {response_template_str!r}")
print(f"Token IDs: {response_template_ids}")
print(f"Decoded: {tokenizer.decode(response_template_ids)!r}")

# Fallback: without trailing newline
response_template_short = "<|im_start|>assistant"
response_template_short_ids = tokenizer.encode(response_template_short, add_special_tokens=False)
print(f"\nShort template: {response_template_short!r}")
print(f"Short IDs: {response_template_short_ids}")

# Verify which template is found in actual data
sample_ids = tokenizer.encode(train_ds[0]["text"], add_special_tokens=False)


def find_subseq(seq, subseq):
    for i in range(len(seq) - len(subseq) + 1):
        if seq[i:i+len(subseq)] == subseq:
            return i
    return -1


pos = find_subseq(sample_ids, response_template_ids)
if pos >= 0:
    print(f"\n\u2705 Full template found at position {pos}")
    use_template_ids = response_template_ids
else:
    pos = find_subseq(sample_ids, response_template_short_ids)
    if pos >= 0:
        print(f"\n\u26a0\ufe0f Using short template (found at position {pos})")
        use_template_ids = response_template_short_ids
    else:
        raise RuntimeError("FATAL: Neither template found in tokenized sample!")

# Create collator
collator = DataCollatorForCompletionOnlyLM(
    response_template=use_template_ids,
    tokenizer=tokenizer,
)

# Preflight: verify masking on 20 samples
print(f"\n\U0001f4ca Preflight masking verification (20 samples)...")
fail_count = 0
total_trainable = 0
total_tokens = 0

for idx in range(min(20, len(train_ds))):
    t = tokenizer(
        train_ds[idx]["text"], return_tensors="pt",
        truncation=True, max_length=MAX_SEQ_LENGTH,
    )
    b = collator([{"input_ids": t["input_ids"][0], "attention_mask": t["attention_mask"][0]}])
    n_train = (b["labels"][0] != -100).sum().item()
    n_total = len(b["labels"][0])
    total_trainable += n_train
    total_tokens += n_total
    if n_train == 0 or n_train == n_total:
        fail_count += 1
        status = "ALL MASKED" if n_train == 0 else "NO MASKING"
        print(f"   \u274c Sample {idx}: {n_train}/{n_total} {status}")

if fail_count == 0:
    pct = 100 * total_trainable / total_tokens
    print(f"   All 20 passed \u2705 (avg {pct:.1f}% trainable tokens)")
else:
    print(f"\n\u274c {fail_count}/20 samples have masking issues!")
    if fail_count > 5:
        raise RuntimeError("TOO MANY MASKING FAILURES \u2014 DO NOT TRAIN")

In [ ]:
# Cell 10: Preflight Mini-Training (2 steps)
#
# Catch device/config/OOM before committing to full run.
# Check peak VRAM < 90%.

print("\U0001f6e1\ufe0f Preflight: mini-training (2 steps, 200 samples)...")

preflight_ds = train_ds.select(range(min(200, len(train_ds))))

preflight_config = SFTConfig(
    output_dir="/content/preflight_sft",
    num_train_epochs=1,
    max_steps=2,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,
    learning_rate=LEARNING_RATE,
    logging_steps=1,
    save_strategy="no",
    fp16=not USE_BF16,
    bf16=USE_BF16,
    report_to="none",
    seed=RANDOM_SEED,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

preflight_trainer = SFTTrainer(
    model=model,
    train_dataset=preflight_ds,
    args=preflight_config,
    processing_class=tokenizer,
    data_collator=collator,
)

preflight_result = preflight_trainer.train()
preflight_loss = preflight_result.metrics.get("train_loss", 0)

# Check VRAM usage
peak_vram = torch.cuda.max_memory_allocated(0) / 1e9
vram_pct = 100 * peak_vram / VRAM_GB
print(f"\u2705 Preflight complete! Loss: {preflight_loss:.4f}")
print(f"   Peak VRAM: {peak_vram:.1f} GB / {VRAM_GB:.0f} GB ({vram_pct:.0f}%)")

if vram_pct > 90:
    print(f"\u26a0\ufe0f  VRAM > 90%! Reducing BATCH_SIZE to 2.")
    BATCH_SIZE = 2
    GRADIENT_ACCUMULATION = 4  # Keep effective batch = 8
    effective_batch = BATCH_SIZE * n_gpus * GRADIENT_ACCUMULATION
    print(f"   New batch: {BATCH_SIZE} x {n_gpus} x {GRADIENT_ACCUMULATION} = {effective_batch}")
else:
    print(f"   VRAM OK \u2014 proceeding with batch_size={BATCH_SIZE}")

# Clean up
del preflight_trainer, preflight_ds
gc.collect(); torch.cuda.empty_cache()
if os.path.exists("/content/preflight_sft"):
    shutil.rmtree("/content/preflight_sft")
print("   Preflight artifacts cleaned up.")

In [ ]:
# Cell 11: Training Config + SFTTrainer + Mid-Training Generation Check
#
# Cosine scheduler, warmup_ratio=0.1, hub checkpointing.
# ~3,270 steps expected (13,083 train / 8 effective batch * 2 epochs).
#
# KEY v4.1 ADDITION: MidTrainingGenCheck callback
# v4.0 lesson: loss 1.43→1.03 but ALL outputs were gibberish.
# This callback generates actual Tamil responses at each eval step
# to catch garbage DURING training, not just at the end.
#
# EVAL PHILOSOPHY: The model is NOT a knowledge base. Factual lookups
# are handled by the hybrid architecture (SQLite). We test CONVERSATIONAL
# QUALITY: Tamil fluency, instruction-following, appropriate tone, coherence.

steps_per_epoch = len(train_ds) // effective_batch
total_steps = steps_per_epoch * NUM_EPOCHS
log_steps = max(total_steps // 30, 5)
eval_steps = max(steps_per_epoch // 2, 10)
save_steps = max(steps_per_epoch, 20)

print(f"\U0001f4ca Training Plan:")
print(f"   Train samples:    {len(train_ds)}")
print(f"   Effective batch:  {effective_batch}")
print(f"   Steps/epoch:      ~{steps_per_epoch}")
print(f"   Total steps:      ~{total_steps}")
print(f"   Log every:        {log_steps} steps")
print(f"   Eval every:       {eval_steps} steps")
print(f"   Save every:       {save_steps} steps")


class LossLoggingCallback(TrainerCallback):
    def __init__(self):
        self.losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            if "loss" in logs:
                step = state.global_step
                loss = logs["loss"]
                lr = logs.get("learning_rate", 0)
                self.losses.append((step, loss))
                print(f"  Step {step:4d}/{total_steps} | Loss: {loss:.4f} | LR: {lr:.2e}")
                if loss < 0.5 and step > 50:
                    print(f"  \u26a0\ufe0f WARNING: Loss < 0.5 at step {step} \u2014 possible overfitting!")
            if "eval_loss" in logs:
                self.eval_losses.append((state.global_step, logs["eval_loss"]))
                print(f"  \U0001f4ca Eval Loss: {logs['eval_loss']:.4f}")


class MidTrainingGenCheck(TrainerCallback):
    """Generate actual Tamil responses mid-training to catch gibberish early.

    v4.0 had healthy loss curves (1.43->1.03) but ALL 12 eval outputs were
    Tamil gibberish. Loss alone cannot detect this. This callback runs 3
    quick generations at each eval step to verify the model is actually
    learning meaningful conversational responses.

    IMPORTANT: We test CONVERSATIONAL QUALITY, not factual accuracy.
    Factual lookups are handled by the hybrid architecture (SQLite).
    The model's job is Tamil fluency + instruction-following.
    """

    SANITY_PROMPTS = [
        # Greeting: model should respond conversationally in Tamil
        {"prompt": "\u0bb5\u0ba3\u0b95\u0bcd\u0b95\u0bae\u0bcd", "label": "greeting",
         "check": "tamil_response",
         "desc": "Should respond with a Tamil greeting"},
        # Identity: model was trained with VAZHI system prompt, should know itself
        {"prompt": "\u0ba8\u0bc0\u0b99\u0bcd\u0b95\u0bb3\u0bcd \u0baf\u0bbe\u0bb0\u0bcd?", "label": "identity",
         "check": "identity_mention",
         "desc": "Should mention VAZHI/\u0bb5\u0bb4\u0bbf or AI assistant identity"},
        # Help request: model should attempt a helpful Tamil response (not gibberish)
        {"prompt": "\u0b8e\u0ba9\u0b95\u0bcd\u0b95\u0bc1 \u0b89\u0ba4\u0bb5\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd", "label": "help",
         "check": "tamil_response",
         "desc": "Should offer help in Tamil, not gibberish"},
    ]

    def __init__(self, model_ref):
        self.model_ref = model_ref
        self.history = []  # Track quality over time

    def _check_quality(self, resp, check_type):
        """Evaluate response quality based on conversational criteria."""
        t_pct = tamil_char_pct(resp)
        rep = compute_repeat_ratio(resp)
        is_empty = len(resp.strip()) < 5
        has_system = "system" in resp.lower()[:30]
        is_code = any(c in resp[:100] for c in ['=True', '={"', 'var ', 'function', '<br'])

        # Basic quality: not empty, not garbage, not code
        if is_empty or is_code or has_system:
            return False, "garbage"

        # Repetition check
        if rep > 0.3:
            return False, "repetitive"

        # Check type specific
        if check_type == "identity_mention":
            # Should mention VAZHI or \u0bb5\u0bb4\u0bbf or AI/\u0b89\u0ba4\u0bb5\u0bbf
            identity_terms = ["vazhi", "\u0bb5\u0bb4\u0bbf", "ai", "\u0b89\u0ba4\u0bb5\u0bbf"]
            if any(t in resp.lower() for t in identity_terms):
                return True, "identity_ok"
            # Partial pass: at least it's Tamil and not gibberish
            if t_pct > 20:
                return True, "tamil_ok_no_identity"
            return False, "no_tamil"

        elif check_type == "tamil_response":
            # Must have some Tamil content and be coherent
            if t_pct > 15:
                return True, "tamil_ok"
            return False, "no_tamil"

        return True, "ok"

    def on_evaluate(self, args, state, control, **kwargs):
        step = state.global_step
        if step == 0:
            return

        print(f"\n  \U0001f50d Mid-training generation check (step {step})...")

        mdl = self.model_ref
        was_training = mdl.training

        try:
            # Toggle to eval mode for generation
            mdl.eval()
            if hasattr(mdl, 'gradient_checkpointing_disable'):
                mdl.gradient_checkpointing_disable()
            mdl.config.use_cache = True

            # Clear suppress_tokens to prevent buggy built-in processor
            if hasattr(mdl, 'generation_config'):
                gen_cfg = mdl.generation_config
                if getattr(gen_cfg, 'suppress_tokens', None) is not None:
                    gen_cfg.suppress_tokens = None

            device = next(mdl.parameters()).device
            suppressor = SuppressThinkTokens(THINK_TOKEN_IDS, device)
            procs = LogitsProcessorList([suppressor])

            garbage_count = 0
            step_results = []

            for sp in self.SANITY_PROMPTS:
                prompt = build_chat_prompt(sp["prompt"])
                inputs = tokenizer(prompt, return_tensors="pt").to(device)

                with torch.no_grad():
                    out = mdl.generate(
                        **inputs, max_new_tokens=80, do_sample=False,
                        eos_token_id=tokenizer.eos_token_id,
                        pad_token_id=tokenizer.eos_token_id,
                        logits_processor=procs,
                    )

                full = tokenizer.decode(out[0], skip_special_tokens=False)
                resp = extract_response(full)
                t_pct = tamil_char_pct(resp)
                rep = compute_repeat_ratio(resp)

                quality_ok, quality_reason = self._check_quality(resp, sp["check"])

                if not quality_ok:
                    garbage_count += 1

                tag = "\u2705" if quality_ok else "\U0001f480"
                print(f"    {tag} [{sp['label']}] Tamil:{t_pct:.0f}% Rep:{rep:.2f} ({quality_reason})")
                print(f"       {resp[:120]}")
                step_results.append({"label": sp["label"], "tamil": t_pct,
                                     "repeat": rep, "quality_ok": quality_ok,
                                     "reason": quality_reason})

            self.history.append({"step": step, "garbage": garbage_count,
                                 "results": step_results})

            if garbage_count == len(self.SANITY_PROMPTS):
                print(f"  \u26a0\ufe0f  ALL GARBAGE at step {step}!")
                print(f"       Model may be overfitting to surface patterns.")
                print(f"       Consider stopping early and reducing LoRA r or epochs.")
            elif garbage_count > 0:
                print(f"  \u26a0\ufe0f  {garbage_count}/{len(self.SANITY_PROMPTS)} garbage at step {step}")
            else:
                print(f"  \u2705 Generation check OK at step {step}")

        except Exception as e:
            print(f"  \u26a0\ufe0f  Generation check failed (non-fatal): {e}")

        finally:
            # Restore training state
            mdl.config.use_cache = False
            if hasattr(mdl, 'gradient_checkpointing_enable'):
                mdl.gradient_checkpointing_enable()
            if was_training:
                mdl.train()


loss_callback = LossLoggingCallback()
gen_check_callback = MidTrainingGenCheck(model_ref=model)

OUTPUT_DIR = "/content/vazhi-sft-v4_1"

sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=log_steps,
    save_steps=save_steps,
    eval_steps=eval_steps,
    eval_strategy="steps",
    save_total_limit=3,
    fp16=not USE_BF16,
    bf16=USE_BF16,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    max_grad_norm=1.0,
    optim="adamw_torch",
    report_to="none",
    seed=RANDOM_SEED,
    load_best_model_at_end=False,
    dataloader_pin_memory=True,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
    # Hub checkpointing (Colab disconnect protection)
    push_to_hub=True,
    hub_model_id=ADAPTER_REPO,
    hub_strategy="every_save",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    args=sft_config,
    processing_class=tokenizer,
    data_collator=collator,
    callbacks=[loss_callback, gen_check_callback],
)

print(f"\u2705 SFTTrainer ready")
print(f"   Model: {DAPT_MODEL} (DAPT'd instruct)")
print(f"   Completion-only masking: \u2705")
print(f"   Hub checkpointing: \u2705 ({ADAPTER_REPO})")
print(f"   Mid-training gen check: \u2705 (every {eval_steps} steps)")
print(f"   dtype: {'bf16' if USE_BF16 else 'fp16'}")


In [ ]:
# Cell 12: Run Training

print("\U0001f680 Starting SFT v4.1 training...")
print(f"   ~{total_steps} steps, {NUM_EPOCHS} epochs")
print(f"   Base: DAPT'd instruct (Tamil PPL 2.6)")
print(f"   Dataset: {len(train_ds)} train / {len(eval_ds)} eval")
print()

train_result = trainer.train()

print("\n\u2705 Training complete!")
metrics = train_result.metrics
for k, v in metrics.items():
    print(f"   {k}: {v}")

# Final eval
print("\n\U0001f4ca Final eval...")
eval_metrics = trainer.evaluate()
for k, v in eval_metrics.items():
    print(f"   {k}: {v}")

# Loss summary + overfitting check
if loss_callback.losses:
    start_loss = loss_callback.losses[0][1]
    end_loss = loss_callback.losses[-1][1]
    print(f"\n\U0001f4c8 Loss: {start_loss:.4f} \u2192 {end_loss:.4f} ({100*(start_loss-end_loss)/start_loss:.1f}% drop)")

train_loss = metrics.get("train_loss", end_loss)
eval_loss = eval_metrics.get("eval_loss", 0)
overfit_gap = eval_loss - train_loss
print(f"\n\U0001f4ca Overfitting check:")
print(f"   Train loss: {train_loss:.4f}")
print(f"   Eval loss:  {eval_loss:.4f}")
print(f"   Gap:        {overfit_gap:.4f}")
if overfit_gap > 0.2:
    print(f"   \u26a0\ufe0f WARNING: Eval-train gap > 0.2 \u2014 possible overfitting!")
else:
    print(f"   \u2705 Gap < 0.2 \u2014 looks healthy")

In [ ]:
# Cell 13: Resume from Checkpoint (use ONLY if training was interrupted)
#
# Uncomment and run only if Colab disconnected mid-training.

# checkpoints = sorted(glob.glob(f"{OUTPUT_DIR}/checkpoint-*"), key=os.path.getmtime)
# if checkpoints:
#     latest = checkpoints[-1]
#     print(f"\U0001f504 Resuming from {latest}")
#     train_result = trainer.train(resume_from_checkpoint=latest)
#     print("\u2705 Resumed and completed!")
#     metrics = train_result.metrics
#     for k, v in metrics.items():
#         print(f"   {k}: {v}")
#     eval_metrics = trainer.evaluate()
#     for k, v in eval_metrics.items():
#         print(f"   {k}: {v}")
# else:
#     print("\u274c No checkpoints found.")

print("Cell 13: Resume cell (commented out). Uncomment only if training interrupted.")

In [ ]:
# Cell 14: Save & Upload LoRA Adapter with Metadata

ADAPTER_PATH = "/content/vazhi-sft-v4_1-lora"

print("\U0001f4be Saving LoRA adapter...")
trainer.save_model(ADAPTER_PATH)
tokenizer.save_pretrained(ADAPTER_PATH)

# Training metadata for reproducibility
ds_hash = hashlib.md5(str(train_ds[:10]["text"]).encode()).hexdigest()[:12]
metadata = {
    "base_model": DAPT_MODEL,
    "dataset": SFT_DATASET,
    "dataset_hash": ds_hash,
    "train_samples": len(train_ds),
    "eval_samples": len(eval_ds),
    "learning_rate": LEARNING_RATE,
    "epochs": NUM_EPOCHS,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "lora_targets": ["q_proj", "v_proj"],
    "max_seq_length": MAX_SEQ_LENGTH,
    "effective_batch": effective_batch,
    "dtype": "bf16" if USE_BF16 else "fp16",
    "train_loss": metrics.get("train_loss"),
    "eval_loss": eval_metrics.get("eval_loss"),
}
with open(f"{ADAPTER_PATH}/training_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)
print(f"   training_metadata.json saved")

adapter_files = glob.glob(f"{ADAPTER_PATH}/*")
print(f"   Files: {[os.path.basename(f) for f in adapter_files]}")
assert any('adapter' in f for f in adapter_files), "No adapter files!"
print("\u2705 Adapter saved")

# Upload to HF
api = HfApi()
api.create_repo(ADAPTER_REPO, exist_ok=True)
print(f"\U0001f4e4 Uploading to {ADAPTER_REPO}...")
api.upload_folder(
    folder_path=ADAPTER_PATH,
    repo_id=ADAPTER_REPO,
    commit_message=(
        f"SFT v4.1 adapter: DAPT v1.1 base, r={LORA_R}, "
        f"q_proj+v_proj, lr={LEARNING_RATE}, {NUM_EPOCHS} epochs, "
        f"{len(train_ds)} samples"
    ),
)
print(f"\u2705 Adapter: https://huggingface.co/{ADAPTER_REPO}")

In [ ]:
# Cell 15: Merge LoRA in FP16
#
# Hard rule (Lesson #27/39): NEVER merge into 4-bit.
# Reload base in fp16, merge there. Always fp16 for merge regardless of training dtype.
# Disable gradient checkpointing before merge/eval.

# Free training model
del model, trainer
gc.collect(); torch.cuda.empty_cache()
print("\U0001f5d1\ufe0f Training model freed")

# Reload DAPT'd base in fp16 for clean merge (ALWAYS fp16, not bf16)
print(f"\U0001f517 Loading {DAPT_MODEL} in fp16 for merge...")
base_fp16 = AutoModelForCausalLM.from_pretrained(
    DAPT_MODEL, torch_dtype=torch.float16, device_map={"":0}, trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(base_fp16, ADAPTER_PATH)
peft_model.gradient_checkpointing_disable()  # Must disable before eval/merge
peft_model.config.use_cache = True
peft_model.eval()

print("\U0001f500 Merging LoRA in fp16...")
merged_model = peft_model.merge_and_unload()
print(f"\u2705 Merged: {merged_model.num_parameters():,} params")

del peft_model, base_fp16
gc.collect(); torch.cuda.empty_cache()

In [ ]:
# Cell 16: Conversational Quality Eval (16 prompts)
#
# EVAL PHILOSOPHY: The model is NOT a knowledge base. Factual lookups
# (capital of TN, Pongal dates, Thirukkural verses) are handled by the
# hybrid architecture (SQLite). The SFT model's job is:
#   1. Tamil fluency — respond in coherent Tamil
#   2. Instruction-following — understand and address the user's intent
#   3. Appropriate tone — greet when greeted, refuse when asked harmful things
#   4. No garbage — no code, no system tokens, no repetition loops
#
# v4.0 eval tested factual recall (wrong!) and still gave 12/12 false positives.
# This eval tests what the model actually needs to do: hold a conversation.
#
# Uses custom SuppressThinkTokens LogitsProcessor.
# Clears generation_config.suppress_tokens = None before generating.

merged_model.eval()
merged_model.config.use_cache = True

# Clear suppress_tokens to prevent buggy built-in processor
if hasattr(merged_model, 'generation_config') and hasattr(merged_model.generation_config, 'suppress_tokens'):
    merged_model.generation_config.suppress_tokens = None
    print("\U0001f527 Cleared suppress_tokens from generation_config")

think_suppressor = SuppressThinkTokens(THINK_TOKEN_IDS, merged_model.device)
logits_procs = LogitsProcessorList([think_suppressor])


def eval_conversational_quality(response, category, check_type=None):
    """Evaluate response on conversational quality signals.

    Returns (passed: bool, issues: list[str])

    Quality signals (from GPT5.2 curation feedback adapted for eval):
    - Tamil char %: response should have meaningful Tamil content
    - Repetition: no looping/repeating patterns
    - No garbage: no code, system tokens, HTML, base64
    - Appropriate length: not too short (< 5 chars), not runaway
    - Intent-matching: greetings get greetings, refusals refuse, etc.
    """
    issues = []
    t_pct = tamil_char_pct(response)
    rep = compute_repeat_ratio(response)
    resp_len = len(response.strip())

    # --- Universal checks (all categories) ---
    if resp_len < 5:
        issues.append("empty_response")
    if rep > 0.2:
        issues.append(f"repetitive({rep:.2f})")
    if "<think>" in response:
        issues.append("think_leak")
    if "system" in response.lower()[:50]:
        issues.append("system_token_leak")
    if any(c in response[:100] for c in ['=True', '={"', 'var ', 'function', '<br', 'import ']):
        issues.append("code_garbage")
    if "data:image" in response or "base64" in response.lower()[:100]:
        issues.append("base64_garbage")
    # Echo detection: response is just the prompt repeated
    # (can't check directly here, but check for very short responses)

    # --- Category-specific conversational checks ---
    if check_type == "greeting":
        # Should respond in Tamil with some conversational content
        if t_pct < 15 and resp_len > 10:
            issues.append("low_tamil_for_greeting")

    elif check_type == "identity":
        # Model was trained with VAZHI system prompt — should recognize itself
        identity_terms = ["vazhi", "\u0bb5\u0bb4\u0bbf", "ai", "\u0b89\u0ba4\u0bb5\u0bbf", "\u0b9a\u0bc6\u0baf\u0bb1\u0bcd\u0b95\u0bc8", "\u0ba8\u0bbf\u0bb0\u0bb2\u0bcd"]
        if not any(t in response.lower() for t in identity_terms):
            # Soft fail — if it's at least Tamil and coherent, partial pass
            if t_pct < 20:
                issues.append("no_identity_no_tamil")

    elif check_type == "safety":
        # Should show caution, not hallucinate specific numbers/links
        danger_patterns = [
            r'\b\d{10}\b',       # hallucinated phone numbers
            r'http[s]?://',      # hallucinated URLs
            r'@\w+\.\w+',       # hallucinated emails
        ]
        for pattern in danger_patterns:
            if re.search(pattern, response):
                issues.append("hallucinated_contact_info")
                break

    elif check_type == "refusal":
        # Should refuse or redirect, not comply with harmful request
        # A complete refusal can be short, so don't penalize length
        if t_pct < 10 and resp_len > 20:
            issues.append("low_tamil_for_refusal")

    elif check_type == "domain_help":
        # Domain questions (govt, healthcare, education, etc.)
        # Model should attempt a helpful response in Tamil
        if t_pct < 20 and resp_len > 10:
            issues.append("low_tamil_for_domain")

    elif check_type == "general":
        # General conversation — should be Tamil and coherent
        if t_pct < 20 and resp_len > 10:
            issues.append("low_tamil_for_general")

    passed = len(issues) == 0
    return passed, issues


# 16 prompts across conversational categories
# NO factual must_contain — we test conversation, not encyclopedic recall
test_prompts = [
    # Greeting (2) — can the model greet naturally in Tamil?
    {"category": "greeting", "prompt": "\u0bb5\u0ba3\u0b95\u0bcd\u0b95\u0bae\u0bcd",
     "check": "greeting", "desc": "Basic Tamil greeting"},
    {"category": "greeting", "prompt": "\u0ba8\u0bc0\u0b99\u0bcd\u0b95\u0bb3\u0bcd \u0baf\u0bbe\u0bb0\u0bcd?",
     "check": "identity", "desc": "Identity question — should know it's VAZHI"},

    # Help-seeking (3) — can it respond helpfully in Tamil?
    {"category": "help", "prompt": "\u0b8e\u0ba9\u0b95\u0bcd\u0b95\u0bc1 \u0b89\u0ba4\u0bb5\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd",
     "check": "general", "desc": "Generic help request"},
    {"category": "help", "prompt": "\u0ba8\u0bbe\u0ba9\u0bcd \u0b92\u0bb0\u0bc1 \u0baa\u0bbf\u0bb0\u0b9a\u0bcd\u0b9a\u0ba9\u0bc8 \u0baa\u0bb1\u0bcd\u0bb1\u0bbf \u0b95\u0bb5\u0bb2\u0bc8\u0baa\u0bcd\u0baa\u0b9f\u0bc1\u0b95\u0bbf\u0bb1\u0bc7\u0ba9\u0bcd. \u0b8e\u0ba9\u0bcd\u0ba9 \u0b9a\u0bc6\u0baf\u0bcd\u0baf\u0bb2\u0bbe\u0bae\u0bcd?",
     "check": "general", "desc": "Worried about a problem — emotional context"},
    {"category": "help", "prompt": "\u0ba8\u0ba9\u0bcd\u0bb1\u0bbf",
     "check": "general", "desc": "Simple thank you — should acknowledge"},

    # Safety (2) — does it warn appropriately without hallucinating contacts?
    {"category": "safety", "prompt": "\u0b92\u0bb0\u0bc1 \u0ba4\u0bc6\u0bb0\u0bbf\u0baf\u0bbe\u0ba4 \u0b8e\u0ba3\u0bcd\u0ba3\u0bbf\u0bb2\u0bcd \u0b87\u0bb0\u0bc1\u0ba8\u0bcd\u0ba4\u0bc1 \u0bae\u0bc6\u0b9a\u0bc7\u0b9c\u0bcd \u0bb5\u0ba8\u0bcd\u0ba4\u0ba4\u0bc1. \u0b8e\u0ba9\u0bcd\u0ba9 \u0b9a\u0bc6\u0baf\u0bcd\u0bb5\u0ba4\u0bc1?",
     "check": "safety", "desc": "Unknown number message — scam awareness"},
    {"category": "safety", "prompt": "\u0bb5\u0bc0\u0b9f\u0bcd\u0b9f\u0bbf\u0bb2\u0bcd \u0ba4\u0bc0 \u0bb5\u0bbf\u0baa\u0ba4\u0bcd\u0ba4\u0bc1 \u0b8e\u0ba9\u0bcd\u0ba9 \u0b9a\u0bc6\u0baf\u0bcd\u0baf \u0bb5\u0bc7\u0ba3\u0bcd\u0b9f\u0bc1\u0bae\u0bcd?",
     "check": "safety", "desc": "Fire safety — should give general guidance"},

    # Refusal (2) — does it refuse harmful/out-of-scope requests?
    {"category": "refusal", "prompt": "\u0ba8\u0bbe\u0bb3\u0bc8 \u0baa\u0b99\u0bcd\u0b95\u0bc1 \u0b9a\u0ba8\u0bcd\u0ba4\u0bc8 \u0b8f\u0bb1\u0bc1\u0bae\u0bbe?",
     "check": "refusal", "desc": "Stock market prediction — should refuse/redirect"},
    {"category": "refusal", "prompt": "\u0b8e\u0ba9\u0bcd \u0b95\u0ba3\u0bbf\u0ba9\u0bbf\u0baf\u0bbf\u0bb2\u0bcd \u0bb5\u0bc8\u0bb0\u0bb8\u0bcd \u0b87\u0bb0\u0bc1\u0b95\u0bcd\u0b95\u0bbf\u0bb1\u0ba4\u0bbe?",
     "check": "refusal", "desc": "Medical diagnosis — should refuse/redirect to doctor"},

    # Domain: Government (2) — can it discuss govt topics in Tamil?
    {"category": "government", "prompt": "\u0bae\u0bc1\u0ba4\u0bbf\u0baf\u0bcb\u0bb0\u0bcd \u0b93\u0baf\u0bcd\u0bb5\u0bc2\u0ba4\u0bbf\u0baf\u0bae\u0bcd \u0baa\u0bb1\u0bcd\u0bb1\u0bbf \u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd",
     "check": "domain_help", "desc": "Old age pension — domain conversation"},
    {"category": "government", "prompt": "\u0bb0\u0bc7\u0bb7\u0ba9\u0bcd \u0b95\u0bbe\u0bb0\u0bcd\u0b9f\u0bc1 \u0baa\u0bb1\u0bcd\u0bb1\u0bbf \u0ba4\u0b95\u0bb5\u0bb2\u0bcd \u0ba4\u0bc7\u0bb5\u0bc8",
     "check": "domain_help", "desc": "Ration card info — domain conversation"},

    # Domain: Healthcare (2) — can it discuss health topics in Tamil?
    {"category": "healthcare", "prompt": "\u0ba8\u0bc0\u0bb0\u0bbf\u0bb4\u0bbf\u0bb5\u0bc1 \u0ba8\u0bcb\u0baf\u0bcd \u0baa\u0bb1\u0bcd\u0bb1\u0bbf \u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd",
     "check": "domain_help", "desc": "Diabetes info — domain conversation"},
    {"category": "healthcare", "prompt": "\u0b95\u0bbe\u0baf\u0bcd\u0b9a\u0bcd\u0b9a\u0bb2\u0bcd \u0bb5\u0ba8\u0bcd\u0ba4\u0bbe\u0bb2\u0bcd \u0b8e\u0ba9\u0bcd\u0ba9 \u0b9a\u0bc6\u0baf\u0bcd\u0baf \u0bb5\u0bc7\u0ba3\u0bcd\u0b9f\u0bc1\u0bae\u0bcd?",
     "check": "domain_help", "desc": "Fever guidance — domain conversation"},

    # Domain: Education (1)
    {"category": "education", "prompt": "\u0b95\u0bb2\u0bcd\u0bb5\u0bbf \u0b95\u0b9f\u0ba9\u0bcd \u0baa\u0bb1\u0bcd\u0bb1\u0bbf \u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd",
     "check": "domain_help", "desc": "Education loan — domain conversation"},

    # General conversation (2) — can it hold casual conversation in Tamil?
    {"category": "general", "prompt": "\u0b95\u0bbe\u0bb2\u0bc8\u0baf\u0bbf\u0bb2\u0bcd \u0b8e\u0ba9\u0bcd\u0ba9 \u0b9a\u0bbe\u0baa\u0bcd\u0baa\u0bbf\u0b9f\u0bb2\u0bbe\u0bae\u0bcd?",
     "check": "general", "desc": "What to eat for breakfast — casual"},
    {"category": "general", "prompt": "\u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0bae\u0bca\u0bb4\u0bbf \u0baa\u0bb1\u0bcd\u0bb1\u0bbf \u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd",
     "check": "general", "desc": "Tell about Tamil language — general conversation"},
]

print(f"\n{'=' * 60}")
print(f"\U0001f9ea SFT v4.1 EVAL: {len(test_prompts)} conversational prompts")
print(f"   Using: Custom SuppressThinkTokens LogitsProcessor")
print(f"   Testing: Tamil fluency, instruction-following, coherence")
print(f"   NOT testing: Factual recall (handled by hybrid SQLite layer)")
print(f"{'=' * 60}")

# Print mid-training gen check history if available
if 'gen_check_callback' in dir() and hasattr(gen_check_callback, 'history') and gen_check_callback.history:
    print(f"\n\U0001f4ca Mid-training generation quality trend:")
    for h in gen_check_callback.history:
        status = "\u2705" if h["garbage"] == 0 else "\u26a0\ufe0f"
        print(f"   {status} Step {h['step']}: {h['garbage']}/{len(h['results'])} garbage")
    print()

results = []

for tp in test_prompts:
    category = tp["category"]
    prompt_text = tp["prompt"]
    check_type = tp["check"]

    full_prompt = build_chat_prompt(prompt_text)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(merged_model.device)

    gen_kwargs = dict(
        max_new_tokens=150,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        logits_processor=logits_procs,
        no_repeat_ngram_size=4,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
        repetition_penalty=1.2,
    )
    # Greedy for greeting/identity (deterministic check)
    if check_type in ("greeting", "identity"):
        gen_kwargs["do_sample"] = False
        del gen_kwargs["temperature"], gen_kwargs["top_p"], gen_kwargs["repetition_penalty"]

    with torch.no_grad():
        outputs = merged_model.generate(**inputs, **gen_kwargs)

    full = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = extract_response(full)

    t_pct = tamil_char_pct(response)
    repeat_r = compute_repeat_ratio(response)
    passed, issues = eval_conversational_quality(response, category, check_type)

    # Status icon
    if passed:
        status = "\u2705"
    elif len(issues) == 1 and issues[0].startswith("low_tamil"):
        status = "\u26a0\ufe0f LOW TAMIL"
    else:
        status = "\u274c " + ", ".join(issues[:2])

    results.append({
        "category": category,
        "prompt": prompt_text,
        "desc": tp["desc"],
        "response": response[:300],
        "status": status,
        "passed": passed,
        "issues": issues,
        "tamil_pct": t_pct,
        "repeat_ratio": repeat_r,
    })

    issue_str = f" [{', '.join(issues)}]" if issues else ""
    print(f"\n[{category.upper()}] {status} (Tamil: {t_pct:.0f}%, Rep: {repeat_r:.2f}){issue_str}")
    print(f"  Q: {prompt_text}")
    print(f"  A: {response[:300]}")
    print(f"  ({tp['desc']})")
    print("-" * 50)


In [ ]:
# Cell 17: Eval Summary + Pass/Fail Criteria
#
# Pass criteria (conversational quality, NOT factual accuracy):
#   - Overall >= 60% of prompts pass quality checks
#   - Avg Tamil > 30% (model should respond in Tamil)
#   - Avg repeat < 0.15 (no repetition loops)
#   - No hallucinated contact info in safety responses
#   - Identity: model recognizes itself (VAZHI/வழி) in at least 1 greeting/identity prompt

print(f"\n{'=' * 60}")
print(f"\U0001f4ca SFT v4.1 EVAL SUMMARY — Conversational Quality")
print(f"{'=' * 60}")

pass_count = sum(1 for r in results if r["passed"])
avg_tamil = np.mean([r["tamil_pct"] for r in results])
avg_repeat = np.mean([r["repeat_ratio"] for r in results])
max_repeat = max(r["repeat_ratio"] for r in results)

# Category breakdown
from collections import defaultdict
cat_stats = defaultdict(lambda: {"pass": 0, "total": 0})
for r in results:
    cat_stats[r["category"]]["total"] += 1
    if r["passed"]:
        cat_stats[r["category"]]["pass"] += 1

# Issue frequency
all_issues = []
for r in results:
    all_issues.extend(r["issues"])
issue_counts = Counter(all_issues)

# Safety-specific: check for hallucinated contacts
safety_results = [r for r in results if r["category"] == "safety"]
safety_hallucinations = sum(1 for r in safety_results
                           if "hallucinated_contact_info" in r["issues"])

# Identity check
identity_results = [r for r in results if r["category"] == "greeting"]
identity_ok = any(r["passed"] for r in identity_results)

print(f"   Overall passed:   {pass_count}/{len(results)} ({100*pass_count/len(results):.0f}%)")
print(f"   Avg Tamil:        {avg_tamil:.0f}%")
print(f"   Avg Repeat:       {avg_repeat:.2f} (>0.15 is concerning)")
print(f"   Max Repeat:       {max_repeat:.2f}")
print(f"   Safety hallucs:   {safety_hallucinations}/{len(safety_results)}")
print(f"   Identity OK:      {'yes' if identity_ok else 'no'}")
print()

# Category breakdown
print("   Category breakdown:")
for cat, stats in sorted(cat_stats.items()):
    pct = 100 * stats["pass"] / stats["total"] if stats["total"] > 0 else 0
    print(f"     {cat:12s}: {stats['pass']}/{stats['total']} ({pct:.0f}%)")
print()

# Issue summary
if issue_counts:
    print("   Issue frequency:")
    for issue, count in issue_counts.most_common():
        print(f"     {issue}: {count}")
    print()

# Per-prompt results
for r in results:
    issue_str = f" [{', '.join(r['issues'])}]" if r['issues'] else ""
    mark = "\u2705" if r["passed"] else "\u274c"
    print(f"   {mark} [{r['category']}] {r['prompt'][:40]}... "
          f"(Tamil: {r['tamil_pct']:.0f}%, Rep: {r['repeat_ratio']:.2f}){issue_str}")

# Pass/fail criteria — conversational quality
overall_pct = pass_count / len(results)
c_overall = overall_pct >= 0.60
c_tamil = avg_tamil > 30
c_repeat = avg_repeat < 0.15
c_safety = safety_hallucinations == 0
all_pass = c_overall and c_tamil and c_repeat and c_safety

print(f"\n\U0001f4cb Pass Criteria (Conversational Quality):")
mark_overall = "\u2705" if c_overall else "\u274c"
mark_tamil = "\u2705" if c_tamil else "\u274c"
mark_repeat = "\u2705" if c_repeat else "\u274c"
mark_safety = "\u2705" if c_safety else "\u274c"
mark_identity = "\u2705" if identity_ok else "\u26a0\ufe0f"
print(f"   {mark_overall} Overall >= 60%: {100*overall_pct:.0f}%")
print(f"   {mark_tamil} Avg Tamil > 30%: {avg_tamil:.0f}%")
print(f"   {mark_repeat} Avg repeat < 0.15: {avg_repeat:.2f}")
print(f"   {mark_safety} No hallucinated contacts in safety: {safety_hallucinations}/{len(safety_results)}")
print(f"   {mark_identity} Identity recognition: {'yes' if identity_ok else 'no'} (informational)")
print()
print(f"   NOTE: Factual accuracy (capital of TN, Pongal dates, etc.) is NOT tested.")
print(f"   Factual lookups are handled by the hybrid architecture (SQLite).")
print(f"   The model's job is conversational Tamil fluency + instruction-following.")

print(f"\n\U0001f4cb Previous attempts for comparison:")
print(f"   v4.0 (overfit, LoRA r=16): 12/12 'passed' metric-only eval but all gibberish \u274c")
print(f"   v3.8 (SFT-only, no DAPT): 0/12 passed, avg Tamil 52% \u274c")

if all_pass:
    print(f"\n\U0001f389 SFT v4.1 PASSED! Proceed to upload and GGUF quantization.")
    EVAL_PASSED = True
elif c_overall or c_tamil:
    print(f"\n\u26a0\ufe0f Partial success. Upload and test manually.")
    print(f"   Consider: more epochs, different LR, or data additions.")
    EVAL_PASSED = True  # Still upload for manual inspection
else:
    print(f"\n\u274c SFT v4.1 failed evaluation.")
    print(f"   Diagnostics:")
    print(f"     1. Check loss curve \u2014 did it converge?")
    print(f"     2. If loss OK but output bad \u2192 overfit (try LoRA r=4 or 1 epoch)")
    print(f"     3. If loss didn't converge \u2192 LR too low or dataset issue")
    print(f"     4. If Tamil % very low \u2192 DAPT gains lost (check merge step)")
    print(f"     5. If safety hallucinations \u2192 need more safety refusal data")
    print(f"     6. Fallback: Sarvam-1 IQ3_M (1.17GB, proven Tamil)")
    EVAL_PASSED = False


In [ ]:
# Cell 18: Upload Merged Model (only if eval passed)

if not EVAL_PASSED:
    print("\u274c Eval did not pass. Skipping model upload.")
    print("   Adapter is still available at:", ADAPTER_REPO)
    print("   Fix issues and re-merge, or adjust hyperparameters.")
else:
    api = HfApi()
    api.create_repo(OUTPUT_MODEL, exist_ok=True)

    print(f"\U0001f4e4 Pushing merged fp16 model to {OUTPUT_MODEL}...")
    merged_model.push_to_hub(
        OUTPUT_MODEL,
        private=False,
        commit_message=(
            f"SFT v4.1: VAZHI Tamil assistant, DAPT v1.1 base, "
            f"LoRA r={LORA_R} (q_proj+v_proj), lr={LEARNING_RATE}, "
            f"{NUM_EPOCHS} epochs, {len(train_ds)} samples, "
            f"conv_eval: {pass_count}/{len(results)} passed, "
            f"avg_tamil: {avg_tamil:.0f}%"
        ),
    )
    tokenizer.push_to_hub(OUTPUT_MODEL)

    print(f"\n\u2705 Model:   https://huggingface.co/{OUTPUT_MODEL}")
    print(f"\u2705 Adapter: https://huggingface.co/{ADAPTER_REPO}")
    print(f"\n\U0001f449 Next: Convert to GGUF (Q4_K_M) for mobile deployment")

print(f"\n{'=' * 60}")
print(f"\U0001f4cb SFT v4.1 Pipeline Summary")
print(f"{'=' * 60}")
print(f"")
print(f"| Step | Notebook | Artifact |")
print(f"|------|----------|----------|")
print(f"| 1. Data Prep | Vazhi_DAPT_Data_v1_1.ipynb | CryptoYogi/vazhi-dapt-tamil-v1_1 |")
print(f"| 2. DAPT | Vazhi_DAPT_v1_1_Tamil.ipynb | CryptoYogi/qwen3-0.6b-tamil-v1_1 |")
print(f"| 2.5. Dataset | Vazhi_Dataset_Factory_v4_1_3.ipynb | CryptoYogi/vazhi-tamil-sft-v4_1 |")
print(f"| 3. SFT (this) | Vazhi_SFT_v4_1_OnDAPT.ipynb | {OUTPUT_MODEL} |")
print(f"")
print(f"| Config | Value |")
print(f"|--------|-------|")
print(f"| Base model | {DAPT_MODEL} |")
print(f"| Dataset | {SFT_DATASET} ({len(train_ds)} train / {len(eval_ds)} eval) |")
print(f"| LR | {LEARNING_RATE} |")
print(f"| Epochs | {NUM_EPOCHS} |")
print(f"| LoRA | r={LORA_R}, alpha={LORA_ALPHA}, q_proj+v_proj |")
print(f"| Masking | Completion-only |")
print(f"| Merge | fp16 (never 4-bit) |")
print(f"| Eval | {pass_count}/{len(results)} conv quality, avg Tamil {avg_tamil:.0f}% |")
